In [ ]:
import gurobipy as gp
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
from sklearn.datasets import make_blobs
from utility import *

Caricamento dell'istanza del problema, calcolo delle distanze tra punti

In [ ]:
test = 2

points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{test}.txt')
n_clusters = [5,5,5,10,5,10,5,5,5,5,4,5,10,5,5]

K = n_clusters[test-1]
N = len(points)

dist = {}

for i in range(N):
    for j in range(N):
        dist[(i,j)] = np.sum((points[i]-points[j])**2)

Definizione delle variabili di decisione, funzione obiettivo e dei vincoli del problema di P-Mediana

In [ ]:
model = gp.Model()
x = model.addVars(N,N, name = "X") #i elemento , j centroide
y = model.addVars(N, vtype = gp.GRB.BINARY, name = "Y")

obj = model.setObjective(x.prod(dist), gp.GRB.MINIMIZE)

model.addConstrs((sum(x[i,j] for j in range(N)) == 1 for i in range(N)))
model.addConstr(sum(y[j] for j in range(N))==K)
model.addConstrs((x[i,j]<=y[j] for i in range(N) for j in range(N)))

model.optimize()


Valutazione della soluzione ottenuta

In [ ]:
#Converto la soluzione nel formato utilizzato dalle euristiche:

centroids = []
for j in range(N):
    if y[j].X == 1:
        centroids.append(j)

sol = np.zeros(N)

for i in range(N):
    for cluster,j in enumerate(centroids):
        if x[i,j].X == 1:
            sol[i] = cluster


print("{}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)